# Classic classification algorithms

We are going to classify our datasets using multiple classification algorithms and evaluate their performances.

In [39]:
# Packages to install
packages_to_install = ['scikit-learn', 'imblearn', 'deslib']

# Check if they are already installed
import importlib
for package in packages_to_install:
    try:
        importlib.import_module(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} is not installed. Installing...")
        !pip install {package}

scikit-learn is not installed. Installing...
imblearn is already installed.
deslib is already installed.


In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

## Load Dataset

In [41]:
x1 = pd.read_csv("../Ressources/Datasets/stand_norm_e1.txt", header=None, delimiter=" ")
x2 = pd.read_csv("../Ressources/Datasets/stand_norm_e2.txt", header=None, delimiter=" ", names=[8, 9, 10, 11, 12, 13, 14])
y = pd.read_csv("../Ressources/Datasets/y2_e1.txt", header=None, delimiter=" ", names=["label"])
x = pd.concat([x1, x2], axis=1)
x = x[1:]
y = y[1:]

x.describe()

,0,1,2,3,4,5,6,8,9,10,11,12,13,14
count,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000,12809.000000
mean,7.715800,105.179340,151.438104,550.188281,1858.821366,0.639516,0.606294,13.894588,151.443231,149.896051,400.836349,1220.861916,0.639516,0.726532
std,13.661444,118.380416,198.978664,596.648195,4357.460170,0.298755,0.355149,25.014059,184.578246,182.014356,427.817220,2481.003246,0.298755,0.402590
min,0.004030,0.497232,1.136887,1.550758,2.000634,0.000000,0.009388,0.008162,1.076040,0.955428,3.392103,7.449574,0.000000,0.006647
25%,0.651308,28.135965,36.205070,160.794149,572.195981,0.407475,0.324020,0.593510,22.236411,36.182643,125.101548,436.947264,0.407475,0.373920
50%,2.563679,73.496119,79.066449,341.937147,1092.008474,0.664512,0.461491,3.255613,83.524212,84.391739,255.453341,768.756167,0.664512,0.567023
75%,9.841018,145.229559,183.952667,722.194942,2030.703113,0.859797,0.844648,17.586211,223.050740,191.915684,518.186018,1335.167249,0.859797,1.063422
max,513.804645,4126.791150,3471.703620,6547.571997,304245.165000,1.932471,1.933941,540.411313,3139.233639,2126.014583,6557.959843,92455.135460,1.932471,1.990678


In [42]:
# Splitting the dataset into training and test set.
x_train, x_test, y_train, y_test = train_test_split(x.values, y.values, test_size = 0.25, random_state=0)

# Feature Scaling
# Adjust the mean to 0 and the standard deviation to 1
st_x = StandardScaler()
x_train = st_x.fit_transform(x_train)
x_test = st_x.transform(x_test)


In [43]:
# Oversample to have same number of samples of each class
smote = SMOTE()
x_train_sampled, y_train_sampled = smote.fit_resample(x_train, y_train)

## Feature extraction : CNN
We are going to extract features from the datasets using the CNN method, knowing that it has the best performances

In [44]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.utils import to_categorical

# Vérification et reformattage de y_train_sampled pour la classification multiclasse
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_sampled.ravel())
y_train_categorical = to_categorical(y_train_encoded)

# Mise à jour de la dernière couche Dense pour qu'elle corresponde au nombre de classes
#num_classes = y_train_categorical.shape[0]
num_classes = y_train_categorical.shape[1]

In [45]:
# Reshape x_train_sampled et x_test_scaled pour l'entrée du CNN
x_train_sampled_cnn = np.expand_dims(x_train_sampled, axis=2)
x_test_scaled_cnn = np.expand_dims(x_test, axis=2)

In [46]:
# Modèle CNN avec trois couches convolutionnelles
cnn_model = Sequential([
    # Augmente la taille de l'input si nécessaire ou ajuste les paramètres
    Conv1D(128, 3, activation='relu', padding='same', input_shape=(x_train_sampled.shape[1], 1)),
    MaxPooling1D(2),
    # Ajout d'une seconde couche convolutive avec padding pour conserver la dimension
    Conv1D(128, 3, activation='relu', padding='same'),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu', padding='same'),
    MaxPooling1D(2),
    Flatten(name='flatten_layer'),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compilation du modèle avec la métrique correcte
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
# Entraînement du modèle
cnn_model.fit(x_train_sampled_cnn, y_train_categorical, epochs=20, batch_size=20, verbose=1)

Epoch 1/20
1128/1128 [==============================] - 14s 11ms/step - loss: 0.5080 - accuracy: 0.7636
Epoch 2/20
1128/1128 [==============================] - 14s 12ms/step - loss: 0.3788 - accuracy: 0.8218
Epoch 3/20
1128/1128 [==============================] - 14s 12ms/step - loss: 0.3531 - accuracy: 0.8347
Epoch 4/20
1128/1128 [==============================] - 10s 9ms/step - loss: 0.3274 - accuracy: 0.8491
Epoch 5/20
1128/1128 [==============================] - 7s 6ms/step - loss: 0.3175 - accuracy: 0.8571
Epoch 6/20
1128/1128 [==============================] - 9s 8ms/step - loss: 0.3008 - accuracy: 0.8675
Epoch 7/20
1128/1128 [==============================] - 7s 6ms/step - loss: 0.2844 - accuracy: 0.8750
Epoch 8/20
1128/1128 [==============================] - 9s 8ms/step - loss: 0.2715 - accuracy: 0.8830
Epoch 9/20
1128/1128 [==============================] - 8s 7ms/step - loss: 0.2575 - accuracy: 0.8869
Epoch 10/20
1128/1128 [==============================] - 8s 7ms/step - loss

In [48]:
# Entraînement du modele avec 30 assages complets de l'ensemble de données d'entraînement &
# 32 batch_size (..) données d'entraînement seront divisées en lots de 32 échantillons.
# Pr chaque lot, les poids du modèle seront MAJ
cnn_model.fit(x_train_sampled_cnn, y_train_categorical, epochs=30, batch_size=32, verbose=1)

Epoch 1/30
705/705 [==============================] - 7s 10ms/step - loss: 0.1295 - accuracy: 0.9440
Epoch 2/30
705/705 [==============================] - 10s 14ms/step - loss: 0.1257 - accuracy: 0.9469
Epoch 3/30
705/705 [==============================] - 10s 14ms/step - loss: 0.1301 - accuracy: 0.9453
Epoch 4/30
705/705 [==============================] - 10s 14ms/step - loss: 0.1186 - accuracy: 0.9476
Epoch 5/30
705/705 [==============================] - 10s 14ms/step - loss: 0.1186 - accuracy: 0.9497
Epoch 6/30
705/705 [==============================] - 5s 7ms/step - loss: 0.1166 - accuracy: 0.9518
Epoch 7/30
705/705 [==============================] - 6s 9ms/step - loss: 0.1116 - accuracy: 0.9534
Epoch 8/30
705/705 [==============================] - 5s 8ms/step - loss: 0.1050 - accuracy: 0.9551
Epoch 9/30
705/705 [==============================] - 5s 7ms/step - loss: 0.1020 - accuracy: 0.9573
Epoch 10/30
705/705 [==============================] - 6s 9ms/step - loss: 0.1051 - accurac

# Traitemet CNN 21.05.2024 proposé

In [49]:
from tensorflow.keras.models import Model
# New modèle pour extraire les caractéristiques de la couche Flatten
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer('flatten_layer').output)

# extrait les caractéristiques pour les ensembles d'entraînement et de test
x_train_sampled_feat_extracted = feature_extractor.predict(x_train_sampled_cnn)
x_test_feat_extracted = feature_extractor.predict(x_test_scaled_cnn)


101/101 [==============================] - 0s 2ms/step


In [77]:
# La méthode score (print_score) n'existe pas pour les objets de type Sequential dans TensorFlow/Keras.
# Dans Keras, l'évaluation des modèles se fait généralement avec les méthodes
# evaluate ou predict pour obtenir des métriques


# Évaluation du modèle
loss, accuracy = cnn_model.evaluate(x_train_sampled, y_train_categorical, verbose=1)

# Affichage de la précision du modèle
print("Accuracy:", accuracy)

705/705 [==============================] - 5s 6ms/step - loss: 0.0541 - accuracy: 0.9787
Accuracy: 0.9786733984947205


## Features Extraction : LDA

In [50]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [51]:
lda = LinearDiscriminantAnalysis(n_components=None)
x_train_sampled_feat_extracted = lda.fit_transform(x_train_sampled, y_train_sampled)
x_test_feat_extracted = lda.transform(x_test)

## Functions print_score, print_accuracy and get_confusion_matrix_and_results

In [52]:
def print_score(model):
    score = model.score(x_train_sampled_feat_extracted, y_train_sampled.ravel())
    print("Score : " + str(score))


def print_accuracy(y):
    print("Accuracy:", accuracy_score(y_test, y))


def get_confusion_matrix_and_results(y):
    # Create confusion matrix
    cm = confusion_matrix(y_test, y)
    cr = classification_report(y_test, y)

    print(cr)
    print(cm)

## Random Forest Algorithm

In [53]:
from sklearn.ensemble import RandomForestClassifier

In [54]:
random_forest = RandomForestClassifier(n_estimators= 20, criterion="entropy", class_weight="balanced")
# random_forest.fit(x_train_sampled, y_train_sampled.ravel())
random_forest.fit(x_train_sampled_feat_extracted, y_train_sampled.ravel())

print_score(random_forest)

Score : 0.993526647157932


In [55]:
# Predicting the test set result
y_pred = random_forest.predict(x_test_feat_extracted)

get_confusion_matrix_and_results(y_pred)

              precision    recall  f1-score   support

         0.0       0.85      0.62      0.72      2500
         1.0       0.31      0.49      0.38       692
         2.0       0.00      0.09      0.01        11

    accuracy                           0.59      3203
   macro avg       0.39      0.40      0.37      3203
weighted avg       0.73      0.59      0.64      3203

[[1539  752  209]
 [ 260  342   90]
 [   5    5    1]]


## Logistic Regression Algorithm

In [56]:
from sklearn.linear_model import LogisticRegression

In [57]:
logistic_regression = LogisticRegression(class_weight="balanced")

logistic_regression.fit(x_train_sampled_feat_extracted, y_train_sampled.ravel())

print_score(logistic_regression)

Score : 0.563935443823712


In [58]:
# Prediction on the test set
y_pred = logistic_regression.predict(x_test_feat_extracted)

get_confusion_matrix_and_results(y_pred)

              precision    recall  f1-score   support

         0.0       0.88      0.63      0.74      2500
         1.0       0.38      0.46      0.42       692
         2.0       0.01      0.45      0.02        11

    accuracy                           0.59      3203
   macro avg       0.43      0.51      0.39      3203
weighted avg       0.77      0.59      0.67      3203

[[1580  502  418]
 [ 205  315  172]
 [   4    2    5]]


## K-Nearest Neighbours (KNN)

In [59]:
from sklearn.neighbors import KNeighborsClassifier

In [60]:
knn = KNeighborsClassifier(n_neighbors=20, weights="uniform")

knn.fit(x_train_sampled_feat_extracted, y_train_sampled.ravel())


print_score(knn)

Score : 0.7141970382193846


In [61]:
# Prediction on the test set
y_pred = knn.predict(x_test_feat_extracted)


get_confusion_matrix_and_results(y_pred)

              precision    recall  f1-score   support

         0.0       0.88      0.63      0.73      2500
         1.0       0.36      0.52      0.42       692
         2.0       0.01      0.36      0.02        11

    accuracy                           0.60      3203
   macro avg       0.42      0.50      0.39      3203
weighted avg       0.76      0.60      0.66      3203

[[1563  644  293]
 [ 212  360  120]
 [   3    4    4]]


## Decision Tree

In [62]:
from sklearn.tree import DecisionTreeClassifier

In [63]:
decision_tree = DecisionTreeClassifier(class_weight="balanced")

decision_tree.fit(x_train_sampled_feat_extracted, y_train_sampled)

print_score(decision_tree)

Score : 1.0


In [64]:
y_pred = decision_tree.predict(x_test_feat_extracted)


get_confusion_matrix_and_results(y_pred)

              precision    recall  f1-score   support

         0.0       0.83      0.56      0.67      2500
         1.0       0.27      0.49      0.35       692
         2.0       0.00      0.09      0.01        11

    accuracy                           0.55      3203
   macro avg       0.37      0.38      0.34      3203
weighted avg       0.71      0.55      0.60      3203

[[1411  908  181]
 [ 277  337   78]
 [   3    7    1]]


# Best classifier selection

In [65]:
from sklearn.ensemble import VotingClassifier

In [66]:
voting_clf = VotingClassifier(estimators=[('rf', random_forest), ('knn', knn), ('arbre decisionnel', decision_tree), ('logistic regression', logistic_regression)], voting='soft')
voting_clf.fit(x_train_sampled_feat_extracted, y_train_sampled)

print_score(voting_clf)

Score : 0.9945020838875588


In [67]:
y_pred = voting_clf.predict(x_test_feat_extracted)

get_confusion_matrix_and_results(y_pred)

              precision    recall  f1-score   support

         0.0       0.85      0.60      0.70      2500
         1.0       0.30      0.50      0.38       692
         2.0       0.00      0.09      0.01        11

    accuracy                           0.58      3203
   macro avg       0.39      0.40      0.36      3203
weighted avg       0.73      0.58      0.63      3203

[[1503  794  203]
 [ 259  349   84]
 [   4    6    1]]


# Boosting
We are going to create a strong model with the Boosting technique using our previous simple models.

In [68]:
from sklearn.ensemble import AdaBoostClassifier

In [69]:
base_models = [
    random_forest,
    logistic_regression,
    decision_tree
]

In [70]:
adaboost_classifiers = []
for base_model in base_models:
    adaboost_classifier = AdaBoostClassifier(base_model, n_estimators=50, learning_rate=1)
    adaboost_classifiers.append(adaboost_classifier)


In [71]:
# Train and evaluate each AdaBoost classifier
for i, adaboost_classifier in enumerate(adaboost_classifiers):
    print("Base model used :", base_models[i])

    # Train the AdaBoost classifier
    adaboost_classifier.fit(x_train_sampled_feat_extracted, y_train_sampled)

    print_score(adaboost_classifier)

    # Make predictions on the test data
    y_pred = adaboost_classifier.predict(x_test_feat_extracted)

    get_confusion_matrix_and_results(y_pred)

    print("")

Base model used : RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       n_estimators=20)
Score : 1.0
              precision    recall  f1-score   support

         0.0       0.87      0.60      0.71      2500
         1.0       0.32      0.53      0.40       692
         2.0       0.01      0.27      0.02        11

    accuracy                           0.59      3203
   macro avg       0.40      0.47      0.38      3203
weighted avg       0.74      0.59      0.64      3203

[[1508  769  223]
 [ 232  368   92]
 [   3    5    3]]

Base model used : LogisticRegression(class_weight='balanced')
Score : 0.5514321184712246
              precision    recall  f1-score   support

         0.0       0.89      0.63      0.74      2500
         1.0       0.38      0.43      0.40       692
         2.0       0.01      0.45      0.02        11

    accuracy                           0.59      3203
   macro avg       0.42      0.51      0.38      3203
weighted 

## DESlib library
We are using the DESlib library to select the best classifier.

In [72]:
# from deslib.des.des_clustering import DESClustering

In [73]:
# des = DESClustering(pool_classifiers=[knn, random_forest, decision_tree])
# des.fit(x_train_sampled_feat_extracted, y_train_sampled.ravel())

# print_score(des)

In [74]:
# y_pred = des.predict(x_test_feat_extracted)


# get_confusion_matrix_and_results(y_pred)